In [69]:
# Cell 1: Imports
# Install required libraries (run in your environment if needed)
# !pip install faker reportlab

from faker import Faker
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from datetime import datetime, timedelta
import random
import os

# Ensure output directory exists
test_dir = "test"
output_dir = "out"
os.makedirs(output_dir, exist_ok=True)

# Initialize Faker
fake = Faker()


In [70]:
# Cell 2: Data
# Unified synthetic data generation with bank-specific customization
def generate_statement_data(bank_name="PNC"):
    """Generate synthetic data with bank-specific customization"""
    start = datetime(2025, 6, 1)
    end = datetime(2025, 6, 30)
    statement_period = f"{start.strftime('%B %d, %Y')} - {end.strftime('%B %d, %Y')}"
    account_type = random.choice(["Standard Checking", "Business Checking"]) if bank_name == "PNC" else random.choice(["Citi Access Checking", "CitiBusiness Checking"])
    account_holder = fake.name()
    account_holder_address_lines = [fake.street_address(), f"{fake.city()}, {fake.state_abbr()} {fake.zipcode()}"]
    customer_iban = fake.iban()
    customer_account_number = fake.bban()
    client_number = fake.ssn()
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%m/%d/%Y")
    
    # Bank-specific settings
    if bank_name == "PNC":
        logo_path = "sample_logos/pnc_logo.png"  # Assume a PNC logo exists
        customer_bank_name = "PNC Bank, National Association"
    else:  # Citibank
        logo_path = "sample_logos/citibank_logo.png"
        customer_bank_name = "Citibank UK Limited"

    # Generate transactions (deposits and withdrawals)
    deposit_descriptions = ["Direct Deposit", "Salary Deposit", "Online Transfer", "Cash Deposit"]
    withdrawal_descriptions = ["Payment", "Grocery Shopping", "Utility Payment", "Online Purchase"]
    transactions = []
    beginning_balance = round(random.uniform(2000, 10000), 2)
    current_balance = beginning_balance
    
    # Generate deposits
    deposits = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(deposit_descriptions)}",
            "amount": f"${round(random.uniform(300, 5000), 2):,.2f}" if bank_name == "PNC" else f"£{round(random.uniform(300, 5000), 2):,.2f}"
        } for _ in range(random.randint(2, 5))
    ]
    
    # Generate withdrawals
    withdrawals = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(withdrawal_descriptions)}",
            "amount": f"-${round(random.uniform(40, 1000), 2):,.2f}" if bank_name == "PNC" else f"-£{round(random.uniform(40, 1000), 2):,.2f}"
        } for _ in range(random.randint(3, 7))
    ]
    
    # Combine and sort transactions by date
    for item in deposits + withdrawals:
        transactions.append({
            "date": item["date"],
            "description": item["description"],
            "debit": f"{item['amount'][1:]}" if "-" in item["amount"] else "",
            "credit": item["amount"] if "-" not in item["amount"] else "",
            "balance": f"{current_balance + (float(item['amount'].replace('$', '').replace('£', '').replace(',', '')) if '-' not in item['amount'] else -float(item['amount'].replace('$', '').replace('£', '').replace('-', '').replace(',', ''))):,.2f}{'$' if bank_name == 'PNC' else '£'}"
        })
        current_balance += float(item['amount'].replace('$', '').replace('£', '').replace(',', '')) if '-' not in item['amount'] else -float(item['amount'].replace('$', '').replace('£', '').replace('-', '').replace(',', ''))
    
    transactions.sort(key=lambda x: datetime.strptime(x["date"], "%m/%d/%Y"))
    
    # Calculate totals
    total_debit = sum(abs(float(item["amount"].replace('$', '').replace('£', '').replace(',', ''))) for item in withdrawals)
    total_credit = sum(float(item["amount"].replace('$', '').replace('£', '').replace(',', '')) for item in deposits)
    total_balance = current_balance
    
    # Summary data
    summary = {
        "opening_balance": f"${beginning_balance:,.2f}" if bank_name == "PNC" else f"£{beginning_balance:,.2f}",
        "total_debit": f"${total_debit:,.2f}" if bank_name == "PNC" else f"£{total_debit:,.2f}",
        "total_credit": f"${total_credit:,.2f}" if bank_name == "PNC" else f"£{total_credit:,.2f}",
        "total": f"${total_balance:,.2f}" if bank_name == "PNC" else f"£{total_balance:,.2f}",
        "transactions_count": len(transactions),
        "overdraft_protection1": "N/A",
        "overdraft_status": "Not enrolled",
        "beginning_balance": f"${beginning_balance:,.2f}" if bank_name == "PNC" else f"£{beginning_balance:,.2f}",
        "deposits_total": f"${total_credit:,.2f}" if bank_name == "PNC" else f"£{total_credit:,.2f}",
        "withdrawals_total": f"-${total_debit:,.2f}" if bank_name == "PNC" else f"-£{total_debit:,.2f}",
        "ending_balance": f"${total_balance:,.2f}" if bank_name == "PNC" else f"£{total_balance:,.2f}",
        "average_balance": f"${round((beginning_balance + total_balance) / 2, 2):,.2f}" if bank_name == "PNC" else f"£{round((beginning_balance + total_balance) / 2, 2):,.2f}",
        "fees": "$0.00" if bank_name == "PNC" else "£0.00",
        "checks_written": "0",
        "pos_transactions": "0",
        "pos_pin_transactions": "0",
        "total_atm_transactions": "0",
        "pnc_atm_transactions": "0",
        "other_atm_transactions": "0",
        "apy_earned": "0.00%",
        "days_in_period": "30",
        "average_collected_balance": f"${round((beginning_balance + total_balance) / 2, 2):,.2f}" if bank_name == "PNC" else f"£{round((beginning_balance + total_balance) / 2, 2):,.2f}",
        "interest_paid_period": "$0.00" if bank_name == "PNC" else "£0.00",
        "interest_paid_ytd": "$0.00" if bank_name == "PNC" else "£0.00",
        "deposits_count": len(deposits),
        "withdrawals_count": len(withdrawals)
    }
    
    return {
        "logo": f"{bank_name.lower()}_logo.png",
        "bank_name": bank_name,
        "logo_path": f"sample_logos/{bank_name.lower()}_logo.png",
        "account_type": account_type,
        "statement_period": statement_period,
        "statement_date": end.strftime("%B %d, %Y"),
        "account_holder": account_holder,
        "account_holder_address_lines": account_holder_address_lines,
        "account_holder_address": ", ".join(account_holder_address_lines),
        "customer_iban": customer_iban,
        "customer_account_number": customer_account_number,
        "client_number": client_number,
        "date_of_birth": date_of_birth,
        "customer_bank_name": customer_bank_name,
        "transactions": transactions,
        "deposits": deposits,
        "withdrawals": withdrawals,
        "show_fee_waiver": random.choice([True, False]),
        "summary": summary
    }

# Generate bank-specific data
pnc_ctx = generate_statement_data("PNC")
citi_ctx = generate_statement_data("Citibank")


In [71]:
# Cell 3a: Citibank Classic

def create_classic_statement(ctx):
    output_file = os.path.join(output_dir, "citibank_statement_classic.pdf")
    c = canvas.Canvas(output_file, pagesize=letter)
    c.setFont("Helvetica", 12)
    
    # Page dimensions
    PAGE_WIDTH, PAGE_HEIGHT = letter
    margin = 0.5 * inch
    usable_width = PAGE_WIDTH - 2 * margin
    y_position = PAGE_HEIGHT - margin
    
    # Helper function to wrap text
    def wrap_text(text, font_size, max_width):
        lines = []
        words = text.split()
        current_line = []
        current_width = 0
        c.setFont("Helvetica", font_size)
        for word in words:
            word_width = c.stringWidth(word + " ", "Helvetica", font_size)
            if current_width + word_width <= max_width:
                current_line.append(word)
                current_width += word_width
            else:
                lines.append(" ".join(current_line))
                current_line = [word]
                current_width = word_width
        if current_line:
            lines.append(" ".join(current_line))
        return lines
    
    # Header
    if os.path.exists(ctx['logo_path']):
        c.drawImage(ctx['logo_path'], PAGE_WIDTH - margin - 150, y_position - 50, width=1.91*inch, height=0.64*inch, mask='auto')


    c.setFillColor(colors.HexColor("#003e7e"))
    c.rect(margin, y_position - 60, usable_width, 4, fill=1)
    y_position -= 72  # Adjust for logo and line
    
    # Bank and Customer Information
    c.setFont("Helvetica-Bold", 9)
    c.drawString(margin, y_position, "Bank information")
    y_position -= 12
    c.setFont("Helvetica", 9)
    c.drawString(margin, y_position, f"Account Provider Name: {ctx['customer_bank_name']}")
    y_position -= 12
    c.drawString(margin, y_position, f"Account Name: {ctx['account_type']}")
    y_position -= 12
    c.drawString(margin, y_position, f"IBAN: {ctx['customer_iban']}")
    y_position -= 12
    c.drawString(margin, y_position, "Country code: GB")
    y_position -= 12
    c.drawString(margin, y_position, f"Check Digits: {ctx['customer_iban'][2:4]}")
    y_position -= 12
    c.drawString(margin, y_position, "Bank code: CITI")
    y_position -= 12
    c.drawString(margin, y_position, f"British bank code (sort code): {ctx['customer_iban'][8:14]}")
    y_position -= 12
    c.drawString(margin, y_position, f"Bank account number: {ctx['customer_account_number']}")
    y_position -= 24  # Space between bank and customer info
    
    c.setFont("Helvetica-Bold", 9)
    c.drawString(margin + usable_width / 2, y_position, "Customer information")
    y_position -= 12
    c.setFont("Helvetica", 9)
    c.drawString(margin + usable_width / 2, y_position, f"Client Name: {ctx['account_holder']}")
    y_position -= 12
    c.drawString(margin + usable_width / 2, y_position, f"Client number ID: {ctx['client_number']}")
    y_position -= 12
    c.drawString(margin + usable_width / 2, y_position, f"Date of birth: {ctx['date_of_birth']}")
    y_position -= 12
    c.drawString(margin + usable_width / 2, y_position, f"Account number: {ctx['customer_account_number']}")
    y_position -= 12
    c.drawString(margin + usable_width / 2, y_position, f"IBAN Bank: {ctx['customer_iban']}")
    y_position -= 12
    c.drawString(margin + usable_width / 2, y_position, f"Bank name: {ctx['customer_bank_name']}")
    y_position -= 24  # Space after customer info
    
    # Important Account Information
    c.setFont("Helvetica", 12)
    c.drawCentredString(PAGE_WIDTH / 2, y_position, "Important Account Information")
    y_position -= 18
    if ctx['account_type'] == "Citi Access Checking":
        info_text = (
            "Effective July 1, 2025, the monthly account fee for Citi Access Checking accounts will increase to £10 unless you maintain a minimum daily balance of £1,500 or have qualifying direct deposits of £500 or more per month. "
            "Starting June 30, 2025, Citibank will introduce real-time transaction alerts for Citi Access Checking accounts via the Citi Mobile UK app. Enable alerts at citibank.co.uk/alerts. "
            "Effective July 15, 2025, Citibank will waive overdraft fees for transactions of £5 or less and cap daily overdraft fees at two per day for Citi Access Checking accounts. "
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly account fee for CitiBusiness Checking accounts will increase to £15 unless you maintain a minimum daily balance of £5,000 or have £2,000 in net purchases on a Citi Business Debit Card per month. "
            "Starting June 30, 2025, Citibank will offer enhanced cash flow tools for CitiBusiness Checking accounts via Citi Online Banking, including automated invoice tracking and payment scheduling. "
            "Effective July 15, 2025, Citibank will reduce domestic BACS transfer fees to £20 for CitiBusiness Checking accounts, down from £25. "
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    c.setFont("Helvetica", 9)
    wrapped_text = wrap_text(info_text.replace("\n", " "), 9, usable_width)
    for line in wrapped_text:
        y_position -= 12
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin
        c.drawString(margin, y_position, line)
    y_position -= 12  # Extra space
    
    # Account Transactions
    c.setFont("Helvetica", 12)
    c.drawCentredString(PAGE_WIDTH / 2, y_position, "Account Transactions")
    y_position -= 18
    c.setFont("Helvetica-Bold", 9)
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['statement_period'])
    y_position -= 12
    c.drawRightString(PAGE_WIDTH - margin, y_position, f"Created on {ctx['statement_date']}")
    y_position -= 12
    
    # Transactions Table (simplified manual drawing)
    col_widths = [0.15 * usable_width, 0.36 * usable_width, 0.12 * usable_width, 0.12 * usable_width, 0.12 * usable_width]
    header_y = y_position
    c.drawString(margin, header_y, "Date")
    c.drawString(margin + col_widths[0], header_y, "Information")
    c.drawString(margin + col_widths[0] + col_widths[1], header_y, "Debit")
    c.drawString(margin + col_widths[0] + col_widths[1] + col_widths[2], header_y, "Credit")
    c.drawString(margin + col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], header_y, "Balance")
    y_position -= 12
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)  # Header line
    
    y_position -= 12
    c.setFont("Helvetica", 9)
    c.drawString(margin, y_position, "")
    c.drawString(margin + col_widths[0], y_position, "Opening balance")
    c.drawRightString(margin + col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], y_position, ctx['summary']['opening_balance'])
    y_position -= 12
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)  # Separator line
    
    for transaction in ctx['transactions'][:5]:  # Fixed to first 5 transactions
        desc = transaction["description"]
        if len(desc) > 25:
            desc = desc[:25] + "..."
        y_position -= 12
        if y_position < margin:  # Page break check
            c.showPage()
            y_position = PAGE_HEIGHT - margin
        c.drawString(margin, y_position, transaction["date"])
        c.drawString(margin + col_widths[0], y_position, desc)
        c.drawRightString(margin + col_widths[0] + col_widths[1] + col_widths[2], y_position, transaction["debit"])
        c.drawRightString(margin + col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], y_position, transaction["credit"])
        c.drawRightString(PAGE_WIDTH - margin, y_position, transaction["balance"])
    
    y_position -= 12
    if y_position < margin:  # Page break check
        c.showPage()
        y_position = PAGE_HEIGHT - margin
    c.drawString(margin, y_position, "")
    c.drawString(margin + col_widths[0], y_position, "Total")
    c.drawRightString(margin + col_widths[0] + col_widths[1] + col_widths[2], y_position, ctx['summary']['total_debit'])
    c.drawRightString(margin + col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], y_position, ctx['summary']['total_credit'])
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['total'])
    y_position -= 12
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)  # Total line
    
    # Notice
    y_position -= 24
    if y_position < margin:  # Page break check
        c.showPage()
        y_position = PAGE_HEIGHT - margin
    c.setFont("Helvetica", 9)
    c.drawCentredString(PAGE_WIDTH / 2, y_position, "This printout is for information purposes only. Your regular account statement of assets takes precedence.")
    y_position -= 24
    
    # Footer
    y_position -= 12
    if y_position < margin:  # Page break check
        c.showPage()
        y_position = PAGE_HEIGHT - margin
    c.setFillColor(colors.HexColor("#003e7e"))
    c.rect(margin, y_position, usable_width, 4, fill=1)
    y_position -= 8
    
    c.setFont("Helvetica", 7)
    c.drawString(margin, y_position, "Citigroup UK Limited is authorised by the Prudential Regulation Authority and regulated by the Financial Conduct Authority and the Prudential Regulation Authority.")
    y_position -= 9
    c.drawString(margin, y_position, "Our firm’s Financial Services Register number is 805574. Citibank UK Limited is a company limited by shares registered in England and Wales with registered address at Citigroup Centre, Canada Square, Canary Wharf, London E14 5LB.")
    y_position -= 9
    c.drawString(margin, y_position, "© All rights reserved Citibank UK Limited 2021. CITI, the Arc Design & Citibank are registered service marks of Citigroup Inc. Calls may be monitored or recorded for training and service quality purposes. PNB FBD 132019.")
    y_position -= 12
    c.drawRightString(PAGE_WIDTH - margin, y_position, "Citibank")
    
    c.save()
    print(f"PDF generated: {output_file}")


In [72]:

# Cell 3b: Placeholder for Chase Classic 

def create_chase_classic_statement(ctx):
    os.makedirs(output_dir, exist_ok=True)  # Changed test_dir to output_dir
    output_file = os.path.join(output_dir, f"chase_classic_statement_{ctx['customer_account_number'][-4:]}.pdf")
    c = canvas.Canvas(output_file, pagesize=letter)

    # ── page / drawing setup ──────────────────────────────────────────
    PAGE_W, PAGE_H = letter
    MARGIN = 0.5 * inch
    usable_w = PAGE_W - 2 * MARGIN
    y = PAGE_H - MARGIN
    ROW_H = 18  # Row height for tables
    HEADER_H = 12
    SECTION_GAP = 22  # Vertical space between sections
    RULE_THICKNESS = 0.4  # All rules 0.4 pt

    # Shortcut
    def str_w(txt, fsize):
        return c.stringWidth(txt, "Helvetica", fsize)

    # Helper – wrap long narrative paragraphs
    def wrap(text, font_size, width):
        c.setFont("Helvetica", font_size)
        out, line = [], []
        used = 0
        for word in text.split():
            w = str_w(word + " ", font_size)
            if used + w > width:
                out.append(" ".join(line))
                line, used = [word], w
            else:
                line.append(word)
                used += w
        if line:
            out.append(" ".join(line))
        return out

    # Helper – draw a thin rule
    def hrule(ypos, x_start, x_end):
        c.setLineWidth(RULE_THICKNESS)
        c.line(x_start, ypos, x_end, ypos)

    # ── (1) HEADER  ───────────────────────────────────────────────────
    if os.path.exists(ctx['logo_path']):
        if ctx['bank_name'] == "PNC":
            c.drawImage(ctx['logo_path'], MARGIN, y - 50, width=0.83*inch, height=0.48*inch, mask='auto')
        elif ctx['bank_name'] in ["Chase", "Citibank"]:
            c.drawImage(ctx['logo_path'], MARGIN, y - 50, width=1.91*inch, height=0.64*inch, mask='auto')
        elif ctx['bank_name'] == "Wells Fargo":
            c.drawImage(ctx['logo_path'], MARGIN, y - 50, width=3.33*inch, height=3.33*inch, mask='auto')
    y -= 60
    c.setFont("Helvetica-Bold", 12)
    c.drawString(MARGIN, y, f"JPMorgan Chase Bank, N.A.")
    c.drawString(MARGIN, y - 15, "PO Box 659754")
    c.drawString(MARGIN, y - 30, "San Antonio, TX 78265-9754")
    c.setFont("Helvetica-Bold", 12)
    c.drawRightString(PAGE_W - MARGIN, y, ctx['statement_period'])
    c.setFont("Helvetica", 12)
    c.drawRightString(PAGE_W - MARGIN, y - 15, f"Account Number: {ctx['customer_account_number']}")
    y_cs = y - 30
    c.setFont("Helvetica-Bold", 12)
    c.drawRightString(PAGE_W - MARGIN, y_cs, "Customer Service Information")
    hrule(y_cs - 3, PAGE_W - MARGIN - 150, PAGE_W - MARGIN)
    hrule(y_cs + 9, PAGE_W - MARGIN - 150, PAGE_W - MARGIN)
    c.setFont("Helvetica", 12)
    cs_items = [
        ("Web site:", "chase.com", 120),
        ("Service Center:", "1-800-242-7338", 70),
        ("Hearing Impaired:", "1-800-242-7383", 60),
        ("Para Espanol:", "1-888-622-4273", 80),
        ("International Calls:", "1-713-262-1679", 60)
    ]
    y_cs -= 15
    for label, value, offset in cs_items:
        c.drawRightString(PAGE_W - MARGIN - offset, y_cs, label)
        c.drawRightString(PAGE_W - MARGIN, y_cs, value)
        y_cs -= 15
    y -= 90

    # ── PAYEE INFO ────────────────────────────────────────────────────
    c.setFont("Helvetica-Bold", 12)
    c.drawString(MARGIN, y, ctx['account_holder'])
    c.drawString(MARGIN, y - 15, ctx['account_holder_address'])
    y -= 40

    # ── IMPORTANT ACCOUNT INFORMATION ─────────────────────────────────
    c.setFont("Helvetica-Bold", 14)
    c.drawString(MARGIN + 12, y, "Important Account Information")
    c.setLineWidth(2)
    hrule(y + 6, MARGIN, PAGE_W - MARGIN)
    y -= HEADER_H
    if ctx['account_type'] == "Chase Total Checking":
        info_text = [
            "Effective July 1, 2025, the monthly service fee for Chase Total Checking accounts will increase to $15 unless you maintain a minimum daily balance of $1,500, have $500 in qualifying direct deposits, or maintain a linked Chase savings account with a balance of $5,000 or more.",
            "Starting June 30, 2025, Chase will introduce real-time transaction alerts for Chase Total Checking accounts via the Chase Mobile app to enhance account monitoring. Enable alerts at chase.com/alerts.",
            "Effective July 15, 2025, Chase will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for Chase Total Checking accounts.",
            "For questions about your account or these changes, please visit chase.com or contact our Customer Service team at 1-800-242-7338, available 24/7."
        ]
    else:
        info_text = [
            "Effective July 1, 2025, the monthly service fee for Chase Business Complete Checking accounts will increase to $20 unless you maintain a minimum daily balance of $2,000, have $2,000 in net purchases on a Chase Business Debit Card, or maintain linked Chase business accounts with a combined balance of $10,000.",
            "Starting June 30, 2025, Chase will offer enhanced cash flow tools for Chase Business Complete Checking accounts via Chase Online, including automated invoice tracking and payment scheduling.",
            "Effective July 15, 2025, Chase will reduce wire transfer fees to $25 for domestic transfers for Chase Business Complete Checking accounts, down from $30.",
            "For questions about your account or these changes, please visit chase.com or contact our Customer Service team at 1-800-242-7338, available 24/7."
        ]
    c.setFont("Helvetica", 12)
    for para in info_text:
        for line in wrap(para, 12, usable_w):
            if y - 15 < MARGIN:
                c.showPage()
                y = PAGE_H - MARGIN
            c.drawString(MARGIN, y, line)
            y -= 15
        y -= 10
    y -= SECTION_GAP

    # ── CHECKING SUMMARY ──────────────────────────────────────────────
    c.setFont("Helvetica-Bold", 14)
    c.drawCentredString(PAGE_W / 2, y, ctx['account_type'])
    y -= 15
    c.drawString(MARGIN + 12, y, "Checking Summary")
    c.setLineWidth(2)
    hrule(y + 6, MARGIN, PAGE_W - MARGIN)
    y -= HEADER_H
    summary_width = 0.60 * usable_w
    summary_x = MARGIN
    col_w = [0.40 * summary_width, 0.30 * summary_width, 0.30 * summary_width]
    col_x = [summary_x]
    for w in col_w[:-1]:
        col_x.append(col_x[-1] + w)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(col_x[0], y, "")
    c.drawString(col_x[1], y, "Instances")
    c.drawString(col_x[2], y, "Amount")
    y -= ROW_H
    summary_items = [
        ("Beginning Balance", "–", ctx['summary']['beginning_balance'].replace('$', '')),
        (f"Deposits and Additions", ctx['summary']['deposits_count'], ctx['summary']['deposits_total'].replace('$', '')),
        (f"Electronic Withdrawals", ctx['summary']['withdrawals_count'], ctx['summary']['withdrawals_total'].replace('$', '')),
        ("Ending Balance", ctx['summary']['transactions_count'], ctx['summary']['ending_balance'].replace('$', ''))
    ]
    c.setFont("Helvetica", 12)
    for label, count, amount in summary_items:
        c.drawString(col_x[0], y, label)
        c.drawString(col_x[1], y, count)
        c.drawString(col_x[2], y, amount)
        y -= ROW_H
    y -= 10
    if ctx.get('show_fee_waiver', False):
        fee_text = (
            "Your monthly service fee was waived because you maintained an average checking balance of $1,500 or had qualifying direct deposits totaling $500 or more during the statement period."
            if ctx['account_type'] == "Chase Total Checking"
            else "Your monthly service fee was waived because you maintained an average checking balance of $10,000 or had $2,500 in qualifying direct deposits during the statement period."
        )
        for line in wrap(fee_text, 12, usable_w):
            c.drawString(MARGIN, y, line)
            y -= 15
    y -= SECTION_GAP

    # ── DEPOSITS AND ADDITIONS ────────────────────────────────────────
    c.setFont("Helvetica-Bold", 14)
    c.drawString(MARGIN + 12, y, "Deposits and Additions")
    c.setLineWidth(2)
    hrule(y + 6, MARGIN, PAGE_W - MARGIN)
    y -= HEADER_H
    data_col_w = [0.15 * usable_w, 0.70 * usable_w, 0.15 * usable_w]
    data_col_x = [MARGIN]
    for w in data_col_w[:-1]:
        data_col_x.append(data_col_x[-1] + w)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(data_col_x[0], y, "Date")
    c.drawString(data_col_x[1], y, "Description")
    c.drawRightString(data_col_x[2] + data_col_w[2] - 2, y, "Amount")
    y -= ROW_H
    c.setLineWidth(2)
    hrule(y + 9, MARGIN, PAGE_W - MARGIN)
    c.setFont("Helvetica", 12)
    deposits = ctx.get('deposits', [])
    if not deposits:
        c.drawString(MARGIN, y, "No deposits for this period.")
        y -= ROW_H
    else:
        for deposit in deposits:
            if y - ROW_H < MARGIN:
                c.showPage()
                y = PAGE_H - MARGIN
                c.setFont("Helvetica-Bold", 12)
                c.drawString(data_col_x[0], y, "Date")
                c.drawString(data_col_x[1], y, "Description")
                c.drawRightString(data_col_x[2] + data_col_w[2] - 2, y, "Amount")
                y -= ROW_H
                c.setLineWidth(2)
                hrule(y + 9, MARGIN, PAGE_W - MARGIN)
            c.setFont("Helvetica", 12)
            c.drawString(data_col_x[0], y, deposit['date'])
            c.drawString(data_col_x[1], y, deposit['description'][:50] + "…" if len(deposit['description']) > 50 else deposit['description'])
            c.drawRightString(data_col_x[2] + data_col_w[2] - 2, y, deposit['amount'].replace('$', ''))
            y -= ROW_H
            c.setLineWidth(2)
            hrule(y + 9, MARGIN, PAGE_W - MARGIN)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(MARGIN, y, "Total Deposits and Additions")
    c.drawRightString(PAGE_W - MARGIN, y, ctx['summary']['deposits_total'].replace('$', ''))
    y -= ROW_H
    y -= SECTION_GAP

    # ── WITHDRAWALS ───────────────────────────────────────────────────
    c.setFont("Helvetica-Bold", 14)
    c.drawString(MARGIN + 12, y, "Withdrawals")
    c.setLineWidth(2)
    hrule(y + 6, MARGIN, PAGE_W - MARGIN)
    y -= HEADER_H
    c.setFont("Helvetica-Bold", 12)
    c.drawString(data_col_x[0], y, "Date")
    c.drawString(data_col_x[1], y, "Description")
    c.drawRightString(data_col_x[2] + data_col_w[2] - 2, y, "Amount")
    y -= ROW_H
    c.setLineWidth(2)
    hrule(y + 9, MARGIN, PAGE_W - MARGIN)
    c.setFont("Helvetica", 12)
    withdrawals = ctx.get('withdrawals', [])
    if not withdrawals:
        c.drawString(MARGIN, y, "No withdrawals for this period.")
        y -= ROW_H
    else:
        for withdrawal in withdrawals:
            if y - ROW_H < MARGIN:
                c.showPage()
                y = PAGE_H - MARGIN
                c.setFont("Helvetica-Bold", 12)
                c.drawString(data_col_x[0], y, "Date")
                c.drawString(data_col_x[1], y, "Description")
                c.drawRightString(data_col_x[2] + data_col_w[2] - 2, y, "Amount")
                y -= ROW_H
                c.setLineWidth(2)
                hrule(y + 9, MARGIN, PAGE_W - MARGIN)
            c.setFont("Helvetica", 12)
            c.drawString(data_col_x[0], y, withdrawal['date'])
            c.drawString(data_col_x[1], y, withdrawal['description'][:50] + "…" if len(withdrawal['description']) > 50 else withdrawal['description'])
            c.drawRightString(data_col_x[2] + data_col_w[2] - 2, y, withdrawal['amount'].replace('$', ''))
            y -= ROW_H
            c.setLineWidth(2)
            hrule(y + 9, MARGIN, PAGE_W - MARGIN)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(MARGIN, y, "Total Electronic Withdrawals")
    c.drawRightString(PAGE_W - MARGIN, y, ctx['summary']['withdrawals_total'].replace('$', ''))
    y -= ROW_H
    y -= SECTION_GAP

    # ── DAILY ENDING BALANCE ──────────────────────────────────────────
    c.setFont("Helvetica-Bold", 14)
    c.drawString(MARGIN + 12, y, "Daily Ending Balance")
    c.setLineWidth(2)
    hrule(y + 6, MARGIN, PAGE_W - MARGIN)
    y -= HEADER_H
    balance_col_w = [0.50 * usable_w, 0.50 * usable_w]
    balance_col_x = [MARGIN]
    for w in balance_col_w[:-1]:
        balance_col_x.append(balance_col_x[-1] + w)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(balance_col_x[0], y, "Date")
    c.drawRightString(balance_col_x[1] + balance_col_w[1] - 2, y, "Amount")
    y -= ROW_H
    c.setFont("Helvetica", 12)
    for balance in ctx.get('daily_balances', []):
        if y - ROW_H < MARGIN:
            c.showPage()
            y = PAGE_H - MARGIN
            c.setFont("Helvetica-Bold", 12)
            c.drawString(balance_col_x[0], y, "Date")
            c.drawRightString(balance_col_x[1] + balance_col_w[1] - 2, y, "Amount")
            y -= ROW_H
        c.setFont("Helvetica", 12)
        c.drawString(balance_col_x[0], y, balance['date'])
        c.drawRightString(balance_col_x[1] + balance_col_w[1] - 2, y, balance['amount'].replace('$', ''))
        y -= ROW_H

    # ── FOOTNOTES AND DISCLOSURES ─────────────────────────────────────
    y -= SECTION_GAP
    c.setFont("Helvetica", 10)
    c.drawString(MARGIN, y, "Disclosures")
    y -= ROW_H
    disclosures_text = (
        "All account transactions are subject to the Chase Deposit Account Agreement, available at chase.com. Interest rates and Annual Percentage Yields (APYs) may change without notice. "
        "For details on overdraft policies and fees, visit chase.com/overdraft or call 1-800-242-7338."
    )
    for line in wrap(disclosures_text, 10, usable_w):
        if y - 12 < MARGIN:
            c.showPage()
            y = PAGE_H - MARGIN
        c.drawString(MARGIN, y, line)
        y -= 12
    c.drawString(MARGIN, y, "JPMorgan Chase Bank, N.A. is a Member FDIC. Equal Housing Lender.")
    y -= 12

    c.save()
    print("PDF generated:", output_file)



In [73]:

# Cell 3c: Placeholder for Wells Fargo Classic

def create_wells_fargo_statement(ctx):
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"wells_fargo_statement_{ctx['customer_account_number'][-4:]}.pdf")
    c = canvas.Canvas(output_file, pagesize=letter)

    # ── page / drawing setup ──────────────────────────────────────────
    PAGE_W, PAGE_H = letter
    MARGIN = 0.5 * inch  # 6% padding approximated as 0.5 inch
    usable_w = PAGE_W - 2 * MARGIN
    y = PAGE_H - MARGIN
    ROW_H = 18  # Row height for tables
    HEADER_H = 12
    SECTION_GAP = 22  # Vertical space between sections
    RULE_THICKNESS = 1  # 1px rule from HTML

    # Shortcut
    def str_w(txt, fsize):
        return c.stringWidth(txt, "Helvetica", fsize)

    # Helper – wrap long narrative paragraphs
    def wrap(text, font_size, width):
        c.setFont("Helvetica", font_size)
        out, line = [], []
        used = 0
        for word in text.split():
            w = str_w(word + " ", font_size)
            if used + w > width:
                out.append(" ".join(line))
                line, used = [word], w
            else:
                line.append(word)
                used += w
        if line:
            out.append(" ".join(line))
        return out

    # Helper – draw a thin rule
    def hrule(ypos, x_start, x_end):
        c.setLineWidth(RULE_THICKNESS)
        c.line(x_start, ypos, x_end, xpos)

    # ── (1) HEADER ───────────────────────────────────────────────────
    c.setFont("Helvetica-Bold", 24)
    c.drawString(MARGIN, y, ctx['account_type'])
    y -= 30
    c.setFont("Helvetica", 13)
    c.drawString(MARGIN, y, f"Account number: {ctx['customer_account_number']} | {ctx['statement_period']}")
    if os.path.exists(ctx['logo_path']):
        c.drawImage(ctx['logo_path'], PAGE_W - MARGIN - 1.91*inch, y, width=1.91*inch, height=0.81*inch, mask='auto')  # 58px height ~ 0.81in
    y -= 36  # Header padding

    # ── (2) ADDRESS & HELP SECTION ────────────────────────────────────
    left_width = 0.60 * usable_w
    right_width = 0.35 * usable_w
    right_x = MARGIN + left_width + 0.05 * usable_w
    c.setFont("Helvetica", 12)
    for line in wrap(ctx['account_holder_address'], 12, left_width):
        c.drawString(MARGIN, y, line)
        y -= 15
    c.setFont("Helvetica-Bold", 12)
    c.drawRightString(PAGE_W - MARGIN, y, "Questions?")
    y -= 15
    c.setFont("Helvetica", 11)
    help_text = [
        "Available by phone 24 hours a day, 7 days a week:",
        "1-800-CALL-WELLS (1-800-225-5935)",
        "",
        "TTY: 1-800-877-4833",
        "En español: 1-877-337-7454",
        "",
        "Online: wellsfargo.com",
        "",
        "Write:",
        "Wells Fargo Bank,",
        "420 Montgomery Street",
        "San Francisco, CA 94104"
    ]
    for line in help_text:
        c.drawRightString(PAGE_W - MARGIN, y, line)
        y -= 13
    c.setLineWidth(1)
    hrule(y + 6, MARGIN, PAGE_W - MARGIN)
    y -= 18

    # ── (3) INTRO BLURB ──────────────────────────────────────────────
    c.setFont("Helvetica-Bold", 16)
    c.drawString(MARGIN, y, "Your Wells Fargo")
    y -= 20
    c.setFont("Helvetica", 11.5)
    intro_text = (
        "It’s a great time to talk with a banker about how Wells Fargo’s accounts "
        "and services can help you stay competitive by saving you time and money. "
        "To find out how we can help, stop by any Wells Fargo location or call us at "
        "the number at the top of your statement."
    )
    for line in wrap(intro_text, 11.5, usable_w):
        c.drawString(MARGIN, y, line)
        y -= 13
    y -= SECTION_GAP

    # ── (4) IMPORTANT ACCOUNT INFORMATION ─────────────────────────────
    c.setFont("Helvetica-Bold", 16)
    c.drawString(MARGIN, y, "Important Account Information")
    y -= 20
    c.setFont("Helvetica", 11.5)
    if ctx['account_type'] == "Everyday Checking":
        info_text = [
            "Effective July 1, 2025, the monthly service fee for Everyday Checking accounts will increase to $12 unless you maintain a minimum daily balance of $500, have $500 in qualifying direct deposits, or maintain a linked Wells Fargo savings account with a balance of $300 or more.",
            "Starting June 30, 2025, Wells Fargo will introduce real-time transaction alerts for Everyday Checking accounts via the Wells Fargo Mobile app. Enable alerts at wellsfargo.com/alerts.",
            "Effective July 15, 2025, Wells Fargo will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for Everyday Checking accounts.",
            "For questions, visit wellsfargo.com or contact our Customer Service at 1-800-225-5935, available 24/7."
        ]
    else:
        info_text = [
            "Effective July 1, 2025, the monthly service fee for Business Checking accounts will increase to $14 unless you maintain a minimum daily balance of $2,500 or have $1,000 in net purchases on a Wells Fargo Business Debit Card per month.",
            "Starting June 30, 2025, Wells Fargo will offer enhanced cash flow tools for Business Checking accounts via Wells Fargo Online Banking, including automated invoice tracking and payment scheduling.",
            "Effective July 15, 2025, Wells Fargo will reduce domestic wire transfer fees to $25 for Business Checking accounts, down from $30.",
            "For questions, visit wellsfargo.com or contact our Customer Service at 1-800-225-5935, available 24/7."
        ]
    for para in info_text:
        for line in wrap(para, 11.5, usable_w):
            if y - 13 < MARGIN:
                c.showPage()
                y = PAGE_H - MARGIN
            c.drawString(MARGIN, y, line)
            y -= 13
        y -= 10
    y -= SECTION_GAP

    # ── (5) ACTIVITY & ROUTING SUMMARIES ──────────────────────────────
    summary_width = 0.48 * usable_w
    summary_x = MARGIN
    right_summary_x = MARGIN + summary_width + 0.02 * usable_w
    c.setFont("Helvetica-Bold", 15)
    c.drawString(summary_x, y, "Activity summary")
    y_summary = y - 20
    c.setFont("Helvetica", 12)
    activity_items = [
        (f"Beginning balance on", ctx['summary']['beginning_balance'].replace('$', '')),
        ("Deposits / Credits", ctx['summary']['deposits_total'].replace('$', '')),
        ("Withdrawals / Debits", ctx['summary']['withdrawals_total'].replace('$', '')),
        ("Ending balance on", ctx['summary']['ending_balance'].replace('$', ''))
    ]
    for label, value in activity_items:
        c.drawString(summary_x + 15, y_summary, label)
        c.drawRightString(summary_x + summary_width - 2, y_summary, value)
        y_summary -= ROW_H
    c.setFont("Helvetica-Bold", 15)
    c.drawString(right_summary_x, y, "Account & routing details")
    y_routing = y - 20
    c.setFont("Helvetica", 10.5)
    routing_items = [
        f"Account number: {ctx['customer_account_number']}",
        ctx['account_holder'],
        "For Direct Deposit and Automatic Payments use Routing Number (RTN): 053000219",
        "For Wire Transfer use Routing Number (RTN): 121000248"
    ]
    for item in routing_items:
        for line in wrap(item, 10.5, summary_width):
            c.drawString(right_summary_x, y_routing, line)
            y_routing -= 12
    y = min(y_summary, y_routing) - SECTION_GAP
    c.setLineWidth(1)
    hrule(y + 6, MARGIN, PAGE_W - MARGIN)
    y -= 12

    # ── (6) TRANSACTION HISTORY ───────────────────────────────────────
    c.setFont("Helvetica-Bold", 15)
    c.drawString(MARGIN, y, "Transaction history")
    y -= 20
    col_w = [0.12 * usable_w, 0.40 * usable_w, 0.16 * usable_w, 0.16 * usable_w, 0.16 * usable_w]
    col_x = [MARGIN]
    for w in col_w[:-1]:
        col_x.append(col_x[-1] + w)
    c.setFillColor(colors.HexColor("#d9d9d9"))
    c.rect(MARGIN, y - 12, usable_w, 12, fill=1, stroke=0)
    c.setFillColor(colors.black)
    c.setFont("Helvetica-Bold", 12)
    hdrs = ["Date", "Description", "Deposits / Credits", "Withdrawals / Debits", "Ending daily balance"]
    for i, h in enumerate(hdrs):
        if i in [2, 3, 4]:
            c.drawRightString(col_x[i] + col_w[i] - 6, y, h)
        else:
            c.drawString(col_x[i], y, h)
    y -= ROW_H
    c.setLineWidth(1)
    hrule(y + 9, MARGIN, PAGE_W - MARGIN)
    c.setFont("Helvetica", 12)
    c.drawString(col_x[0], y, "")
    c.drawString(col_x[1], y, "Opening balance")
    c.drawRightString(col_x[4] + col_w[4] - 6, y, ctx['summary']['beginning_balance'].replace('$', ''))
    y -= ROW_H
    hrule(y + 9, MARGIN, PAGE_W - MARGIN)
    trans_limit = min(10, len(ctx['transactions']))
    for t in ctx['transactions'][:trans_limit]:
        if y - ROW_H < MARGIN:
            c.showPage()
            y = PAGE_H - MARGIN
            c.setFillColor(colors.HexColor("#d9d9d9"))
            c.rect(MARGIN, y - 12, usable_w, 12, fill=1, stroke=0)
            c.setFillColor(colors.black)
            c.setFont("Helvetica-Bold", 12)
            for i, h in enumerate(hdrs):
                if i in [2, 3, 4]:
                    c.drawRightString(col_x[i] + col_w[i] - 6, y, h)
                else:
                    c.drawString(col_x[i], y, h)
            y -= ROW_H
            hrule(y + 9, MARGIN, PAGE_W - MARGIN)
        c.setFont("Helvetica", 12)
        desc = t["description"][:30] + "…" if len(t["description"]) > 30 else t["description"]
        c.drawString(col_x[0], y, t["date"])
        c.drawString(col_x[1], y, desc)
        c.drawRightString(col_x[2] + col_w[2] - 6, y, t["credit"].replace('$', ''))
        c.drawRightString(col_x[3] + col_w[3] - 6, y, t["debit"].replace('$', ''))
        c.drawRightString(col_x[4] + col_w[4] - 6, y, t["balance"].replace('$', ''))
        y -= ROW_H
        hrule(y + 9, MARGIN, PAGE_W - MARGIN)
    c.setFont("Helvetica-Bold", 12)
    c.drawString(col_x[0], y, "")
    c.drawString(col_x[1], y, "Total")
    c.drawRightString(col_x[2] + col_w[2] - 6, y, ctx['summary']['deposits_total'].replace('$', ''))
    c.drawRightString(col_x[3] + col_w[3] - 6, y, ctx['summary']['withdrawals_total'].replace('$', ''))
    c.drawRightString(col_x[4] + col_w[4] - 6, y, ctx['summary']['ending_balance'].replace('$', ''))
    y -= ROW_H

    # ── (7) FOOTNOTES AND DISCLOSURES ─────────────────────────────────
    y -= SECTION_GAP
    c.setFont("Helvetica", 10)
    c.drawString(MARGIN, y, "Disclosures")
    y -= 12
    disclosures_text = (
        "All account transactions are subject to the Wells Fargo Deposit Account Agreement, available at wellsfargo.com. Interest rates and Annual Percentage Yields (APYs) may change without notice. "
        "For details on overdraft policies and fees, visit wellsfargo.com/overdraft or call 1-800-225-5935."
    )
    for line in wrap(disclosures_text, 10, usable_w):
        if y - 12 < MARGIN:
            c.showPage()
            y = PAGE_H - MARGIN
        c.drawString(MARGIN, y, line)
        y -= 12
    c.drawString(MARGIN, y, "© 2025 Wells Fargo Bank, N.A. All rights reserved. Member FDIC.")
    y -= 12

    c.save()
    print("PDF generated:", output_file)



In [74]:
# Cell 3d: PNC Classic

def create_pnc_classic_statement(ctx):
    output_file = os.path.join(output_dir, f"pnc_classic_statement_{ctx['customer_account_number'][-4:]}.pdf")
    c = canvas.Canvas(output_file, pagesize=letter)
    c.setFont("Helvetica", 12)  # Match HTML font
    
    # Page dimensions
    PAGE_WIDTH, PAGE_HEIGHT = letter
    margin = 0.5 * inch
    usable_width = PAGE_WIDTH - 2 * margin
    y_position = PAGE_HEIGHT - margin - 24  # Adjust for top padding
    
    # Helper function to wrap text
    def wrap_text(text, font_size, max_width):
        lines = []
        words = text.split()
        current_line = []
        current_width = 0
        c.setFont("Helvetica", font_size)
        for word in words:
            word_width = c.stringWidth(word + " ", "Helvetica", font_size)
            if current_width + word_width <= max_width:
                current_line.append(word)
                current_width += word_width
            else:
                lines.append(" ".join(current_line))
                current_line = [word]
                current_width = word_width
        if current_line:
            lines.append(" ".join(current_line))
        return lines
    
    # Header
    c.setFont("Helvetica", 22)
    c.drawString(margin, y_position, f"{ctx['account_type']} Statement")
    y_position -= 18
    c.setFont("Helvetica", 10.5)
    c.drawString(margin, y_position, "PNC Bank")
    if os.path.exists(ctx['logo_path']):
        c.drawImage(ctx['logo_path'], PAGE_WIDTH - margin - 60, y_position + 18, width=0.83*inch, height=0.48*inch, mask='auto')
    else:
        print(f"Warning: Logo file {ctx['logo_path']} not found.")
    y_position -= 30  # Adjusted for tighter spacing
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)  # Rule
    
    # Customer & Contact Row
    y_position -= 14
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, f"{ctx['statement_period']}")
    y_position -= 14
    c.drawString(margin, y_position, ctx['account_holder'])
    y_position -= 14
    c.setFont("Helvetica", 12)
    address_lines = wrap_text(ctx['account_holder_address'], 12, usable_width / 2)  # Limit to half the usable width
    for line in address_lines:
        c.drawString(margin, y_position, line)
        y_position -= 12
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin - 24
            c.setFont("Helvetica", 12)
    
    # Right Info Block
    right_x = PAGE_WIDTH - margin - 250  # Adjusted for more space
    c.setFont("Helvetica", 10.5)
    y_position_right = y_position + 8
    c.drawString(right_x, y_position_right, f"Primary account number: {ctx['customer_account_number']}")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "Page 1 of 1")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "Number of enclosures: 0")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "24-hour Online & Mobile Banking at pnc.com")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "Customer service: 1-888-PNC-BANK")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "Mon-Fri 7 AM–10 PM ET • Sat-Sun 8 AM–5 PM ET")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "Spanish: 1-866-HOLA-PNC")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "Write: PO Box 609, Pittsburgh PA 15230-9738")
    y_position_right -= 12
    c.drawString(right_x, y_position_right, "TTY: 1-800-531-1648")
    y_position -= max(14, y_position_right - y_position)  # Align with lowest point
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)
    
    # Important Account Information
    y_position -= 14
    c.setFont("Helvetica", 11)
    c.drawString(margin, y_position, "Important Account Information")
    y_position -= 14
    c.setFont("Helvetica", 12)
    if ctx['account_type'] == "Standard Checking":
        info_text = (
            "Effective July 1, 2025, the monthly service fee will be $10 unless you maintain a minimum daily balance of $1,500 or have qualifying direct deposits of $500 or more per month. "
            "Starting June 30, 2025, PNC will introduce real-time transaction alerts for Standard Checking accounts via the PNC Mobile app. Enable alerts at pnc.com/alerts. "
            "Effective July 15, 2025, PNC will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for Standard Checking accounts. "
            "For questions, visit pnc.com or contact our Customer Service at 1-888-PNC-BANK, available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly service fee for Business Checking accounts will be $15 unless you maintain a minimum daily balance of $5,000 or have $2,000 in net purchases on a PNC Business Debit Card per month. "
            "Starting June 30, 2025, PNC will offer enhanced cash flow tools for Business Checking accounts via PNC Online Banking, including automated invoice tracking and payment scheduling. "
            "Effective July 15, 2025, PNC will reduce domestic wire transfer fees to $25 for Business Checking accounts, down from $30. "
            "For questions, visit pnc.com or contact our Customer Service at 1-888-PNC-BANK, available 24/7."
        )
    wrapped_text = wrap_text(info_text, 12, usable_width)
    for line in wrapped_text:
        y_position -= 12
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin - 24
            c.setFont("Helvetica", 12)
        c.drawString(margin, y_position, line)
    y_position -= 12
    c.drawString(margin, y_position, "Questions? Visit any PNC branch or call 1-888-762-2265 (24/7).")
    y_position -= 14
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)
    
    # Account Summary
    y_position -= 14
    c.setFont("Helvetica", 15)
    c.drawString(margin, y_position, f"{ctx['account_type']} Summary")
    y_position -= 14
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, f"Account number: {ctx['customer_account_number']}")
    y_position -= 12
    c.drawString(margin, y_position, f"Overdraft Protection Provided By: {ctx['summary']['overdraft_protection1']}")
    y_position -= 12
    c.setFont("Helvetica-Bold", 12)
    c.drawString(margin, y_position, f"Overdraft Coverage: Your account is {ctx['summary']['overdraft_status']}.")
    y_position -= 12
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, f"{ctx['account_holder']}")
    y_position -= 12
    address_lines = wrap_text(ctx['account_holder_address'], 12, usable_width)
    for line in address_lines:
        c.drawString(margin, y_position, line)
        y_position -= 12
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin - 24
            c.setFont("Helvetica", 12)
    y_position -= 14
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)
    
    # Balance / Transaction / Interest Summaries
    y_position -= 14
    c.setFont("Helvetica", 13)
    c.drawString(margin, y_position, "Balance Summary")
    y_position -= 10
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, "Beginning balance")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['beginning_balance'])
    y_position -= 12
    c.drawString(margin, y_position, "Deposits & other additions")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['deposits_total'])
    y_position -= 12
    c.drawString(margin, y_position, "Checks & other deductions")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['withdrawals_total'])
    y_position -= 12
    c.drawString(margin, y_position, "Ending balance")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['ending_balance'])
    y_position -= 12
    c.drawString(margin, y_position, "Average monthly balance")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['average_balance'])
    y_position -= 12
    c.drawString(margin, y_position, "Charges & fees")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['fees'])
    y_position -= 20
    if y_position < margin:
        c.showPage()
        y_position = PAGE_HEIGHT - margin - 24
        c.setFont("Helvetica", 12)
    
    c.setFont("Helvetica", 13)
    c.drawString(margin, y_position, "Transaction Summary")
    y_position -= 10
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, "Checks paid/written")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['checks_written'])
    y_position -= 12
    c.drawString(margin, y_position, "Check-card POS transactions")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['pos_transactions'])
    y_position -= 12
    c.drawString(margin, y_position, "Check-card/virtual POS PIN txn")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['pos_pin_transactions'])
    y_position -= 12
    c.drawString(margin, y_position, "Total ATM transactions")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['total_atm_transactions'])
    y_position -= 12
    c.drawString(margin, y_position, "PNC Bank ATM transactions")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['pnc_atm_transactions'])
    y_position -= 12
    c.drawString(margin, y_position, "Other Bank ATM transactions")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['other_atm_transactions'])
    y_position -= 20
    if y_position < margin:
        c.showPage()
        y_position = PAGE_HEIGHT - margin - 24
        c.setFont("Helvetica", 12)
    
    c.setFont("Helvetica", 13)
    c.drawString(margin, y_position, "Interest Summary")
    y_position -= 10
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, "APY earned")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['apy_earned'])
    y_position -= 12
    c.drawString(margin, y_position, "Days in period")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['days_in_period'])
    y_position -= 12
    c.drawString(margin, y_position, "Avg collected balance")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['average_collected_balance'])
    y_position -= 12
    c.drawString(margin, y_position, "Interest paid this period")
    c.drawRightString(PAGE_WIDTH - margin, y_position, ctx['summary']['interest_paid_period'])
    y_position -= 12
    c.setFont("Helvetica", 10.5)
    c.drawString(margin, y_position, f"YTD interest paid: {ctx['summary']['interest_paid_ytd']}")
    y_position -= 20
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)
    
    # Activity Detail
    y_position -= 14
    c.setFont("Helvetica", 15)
    c.drawString(margin, y_position, "Activity Detail")
    y_position -= 14
    
    c.setFont("Helvetica", 13)
    c.drawString(margin, y_position, "Deposits & Other Additions")
    y_position -= 10
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, "Date")
    c.drawString(margin + 0.15 * usable_width, y_position, "Amount")
    c.drawString(margin + 0.35 * usable_width, y_position, "Description")
    y_position -= 12
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)
    for deposit in ctx['deposits'][:3]:  # Fixed to first 3 deposits
        y_position -= 12
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin - 24
            c.setFont("Helvetica", 12)
        c.drawString(margin, y_position, deposit["date"])
        c.drawRightString(margin + 0.35 * usable_width, y_position, deposit["amount"])
        c.drawString(margin + 0.35 * usable_width, y_position, deposit["description"])
    y_position -= 12
    c.setFont("Helvetica", 10.5)
    c.drawString(margin, y_position, f"There are {ctx['summary']['deposits_count']} deposits totaling {ctx['summary']['deposits_total']}.")
    y_position -= 20
    
    c.setFont("Helvetica", 13)
    c.drawString(margin, y_position, "Checks & Other Deductions")
    y_position -= 10
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, "Date")
    c.drawString(margin + 0.15 * usable_width, y_position, "Amount")
    c.drawString(margin + 0.35 * usable_width, y_position, "Description")
    y_position -= 12
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)
    for withdrawal in ctx['withdrawals'][:3]:  # Fixed to first 3 withdrawals
        y_position -= 12
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin - 24
            c.setFont("Helvetica", 12)
        c.drawString(margin, y_position, withdrawal["date"])
        c.drawRightString(margin + 0.35 * usable_width, y_position, withdrawal["amount"])
        c.drawString(margin + 0.35 * usable_width, y_position, withdrawal["description"])
    y_position -= 12
    c.setFont("Helvetica", 10.5)
    c.drawString(margin, y_position, f"There are {ctx['summary']['withdrawals_count']} withdrawals totaling {ctx['summary']['withdrawals_total']}.")
    y_position -= 20
    c.line(margin, y_position, PAGE_WIDTH - margin, y_position)
    
    # Disclosures
    y_position -= 14
    c.setFont("Helvetica", 11)
    c.drawString(margin, y_position, "Disclosures")
    y_position -= 14
    c.setFont("Helvetica", 12)
    disclosures_text = (
        "All account transactions are subject to the PNC Consumer Funds Availability Policy and Account Agreement, available at pnc.com. "
        "Interest rates and Annual Percentage Yields (APYs) may change without notice. For overdraft information, visit pnc.com/overdraft or call 1-888-762-2265."
    )
    wrapped_disclosures = wrap_text(disclosures_text, 12, usable_width)
    for line in wrapped_disclosures:
        y_position -= 12
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin - 24
            c.setFont("Helvetica", 12)
        c.drawString(margin, y_position, line)
    y_position -= 12
    c.drawString(margin, y_position, "PNC Bank, National Association, Member FDIC • Equal Housing Lender.")
    
    c.save()
    print(f"PDF generated: {output_file}")

In [75]:
# Cell 4: Dynamic Statement Generator
import json
import os
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

# Registry for section types
section_registry = {}

# Base Section class
class Section:
    def render(self, canvas, y_position, ctx, config, page_width, page_height, margin):
        pass

# Header Section
class HeaderSection(Section):
    def render(self, canvas, y_position, ctx, config, page_width, page_height, margin):
        canvas.setFont(config.get("font", "Helvetica-Bold"), config.get("font_size", 22))
        title = config.get("title", "{account_type} Statement").format(**ctx)
        canvas.drawString(margin, y_position, title)
        logo_path = ctx.get("logo_path", "")
        if config.get("show_logo", False) and os.path.exists(logo_path):
            canvas.drawImage(logo_path, page_width - margin - config.get("logo_width", 1.91*inch),
                             y_position - config.get("logo_height", 0.64*inch),
                             width=config.get("logo_width", 1.91*inch),
                             height=config.get("logo_height", 0.64*inch), mask='auto')
        return y_position - config.get("spacing", 30)

# Account Info Section
class AccountInfoSection(Section):
    def render(self, canvas, y_position, ctx, config, page_width, page_height, margin):
        usable_width = page_width - 2 * margin
        canvas.setFont(config.get("font", "Helvetica"), config.get("font_size", 12))
        layout = config.get("layout", "single_column")
        fields = config.get("fields", [])
        y_pos = y_position
        if layout == "two_column":
            left_x, right_x = margin, margin + usable_width / 2
            for i, field in enumerate(fields):
                value = ctx.get(field, "") if field in ctx else ctx["summary"].get(field, "")
                text = f"{field.replace('_', ' ').title()}: {value}"
                x_pos = left_x if i % 2 == 0 else right_x
                canvas.drawString(x_pos, y_pos, text)
                if i % 2 == 1:
                    y_pos -= config.get("line_spacing", 12)
                if y_pos < margin:
                    canvas.showPage()
                    y_pos = page_height - margin
                    canvas.setFont(config.get("font", "Helvetica"), config.get("font_size", 12))
        else:
            for field in fields:
                value = ctx.get(field, "") if field in ctx else ctx["summary"].get(field, "")
                canvas.drawString(margin, y_pos, f"{field.replace('_', ' ').title()}: {value}")
                y_pos -= config.get("line_spacing", 12)
                if y_pos < margin:
                    canvas.showPage()
                    y_pos = page_height - margin
                    canvas.setFont(config.get("font", "Helvetica"), config.get("font_size", 12))
        return y_pos - config.get("section_spacing", 20)

# Transaction Table Section
class TransactionTableSection(Section):
    def render(self, canvas, y_position, ctx, config, page_width, page_height, margin):
        usable_width = page_width - 2 * margin
        columns = config.get("columns", [])
        col_widths = [c.get("width", 1/len(columns)) * usable_width for c in columns]
        col_x = [margin]
        for w in col_widths[:-1]:
            col_x.append(col_x[-1] + w)
        
        # Header
        canvas.setFont(config.get("header_font", "Helvetica-Bold"), config.get("header_font_size", 12))
        for i, col in enumerate(columns):
            canvas.drawString(col_x[i], y_position, col["label"] if col.get("align", "left") == "left" else "")
            if col.get("align", "left") == "right":
                canvas.drawRightString(col_x[i] + col_widths[i] - 6, y_position, col["label"])
        y_position -= config.get("row_height", 12)
        canvas.setLineWidth(config.get("line_thickness", 1))
        canvas.line(margin, y_position, page_width - margin, y_position)
        
        # Data
        canvas.setFont(config.get("font", "Helvetica"), config.get("font_size", 10))
        for row in ctx.get("transactions", [])[:config.get("row_limit", 10)]:
            y_position -= config.get("row_height", 12)
            if y_position < margin:
                canvas.showPage()
                y_position = page_height - margin
                canvas.setFont(config.get("header_font", "Helvetica-Bold"), config.get("header_font_size", 12))
                for i, col in enumerate(columns):
                    canvas.drawString(col_x[i], y_position, col["label"] if col.get("align", "left") == "left" else "")
                    if col.get("align", "left") == "right":
                        canvas.drawRightString(col_x[i] + col_widths[i] - 6, y_position, col["label"])
                y_position -= config.get("row_height", 12)
                canvas.setLineWidth(config.get("line_thickness", 1))
                canvas.line(margin, y_position, page_width - margin, y_position)
                canvas.setFont(config.get("font", "Helvetica"), config.get("font_size", 10))
            
            for i, col in enumerate(columns):
                value = row.get(col["field"], "")
                if col.get("align", "left") == "right":
                    canvas.drawRightString(col_x[i] + col_widths[i] - 6, y_position, value)
                else:
                    canvas.drawString(col_x[i], y_position, value[:30] + "..." if len(value) > 30 else value)
        
        # Totals
        y_position -= config.get("row_height", 12)
        canvas.setFont(config.get("header_font", "Helvetica-Bold"), config.get("header_font_size", 12))
        canvas.drawString(margin, y_position, "Total")
        for i, col in enumerate(columns):
            if col["field"] in ["debit", "credit"]:
                canvas.drawRightString(col_x[i] + col_widths[i] - 6, y_position, ctx["summary"][f"total_{col['field']}"])
        y_position -= config.get("row_height", 12)
        canvas.line(margin, y_position, page_width - margin, y_position)
        return y_position - config.get("section_spacing", 20)

# Text Block Section (e.g., Important Information, Disclosures)
class TextBlockSection(Section):
    def render(self, canvas, y_position, ctx, config, page_width, page_height, margin):
        usable_width = page_width - 2 * margin
        canvas.setFont(config.get("font", "Helvetica"), config.get("font_size", 12))
        text = config.get("text", "").format(**ctx)
        lines = []
        current_line = []
        current_width = 0
        for word in text.split():
            word_width = canvas.stringWidth(word + " ", config.get("font", "Helvetica"), config.get("font_size", 12))
            if current_width + word_width <= usable_width:
                current_line.append(word)
                current_width += word_width
            else:
                lines.append(" ".join(current_line))
                current_line = [word]
                current_width = word_width
        if current_line:
            lines.append(" ".join(current_line))
        
        for line in lines:
            y_position -= config.get("line_spacing", 12)
            if y_position < margin:
                canvas.showPage()
                y_position = page_height - margin
                canvas.setFont(config.get("font", "Helvetica"), config.get("font_size", 12))
            canvas.drawString(margin, y_position, line)
        return y_position - config.get("section_spacing", 20)

# Register sections
section_registry = {
    "header": HeaderSection,
    "account_info": AccountInfoSection,
    "transactions": TransactionTableSection,
    "text_block": TextBlockSection
}

def create_dynamic_statement(ctx, config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    output_file = os.path.join("out", f"{ctx['bank_name'].lower()}_dynamic_statement_{ctx['customer_account_number'][-4:]}.pdf")
    c = canvas.Canvas(output_file, pagesize=letter)
    
    PAGE_WIDTH, PAGE_HEIGHT = letter
    margin = config.get("margin", 0.5 * inch)
    y_position = PAGE_HEIGHT - margin
    
    for section_config in config["sections"]:
        section_type = section_config.get("type")
        if section_type not in section_registry:
            print(f"Warning: Unknown section type {section_type}")
            continue
        section = section_registry[section_type]()
        y_position = section.render(c, y_position, ctx, section_config, PAGE_WIDTH, PAGE_HEIGHT, margin)
        if y_position < margin:
            c.showPage()
            y_position = PAGE_HEIGHT - margin
    
    c.save()
    print(f"PDF generated: {output_file}")

# Example usage
# Assumes pnc_ctx and citi_ctx are defined from previous cells
# create_dynamic_statement(pnc_ctx, "statement_config.json")
# create_dynamic_statement(citi_ctx, "statement_config.json")
